### Dashboard

W tym notebooku przedstawiono proces tworzenia interaktywnego dashboardu za pomocą biblioteki Dash oraz Plotly. Dashboard ten umożliwia wizualizację danych pogodowych i energetycznych dla różnych miast w Europie. Celem projektu jest analiza produkcji energii oraz warunków pogodowych w różnych lokalizacjach, a także przedstawienie szacowanego zapotrzebowania na energię dla domów o różnych rozmiarach, ogrzewanych za pomocą pomp ciepła.


##### Opis kroków

1. Importowanie bibliotek: Zaimportowane zostały niezbędne biblioteki, takie jak pandas do manipulacji danymi, dash do tworzenia interaktywnych aplikacji internetowych oraz plotly do tworzenia wykresów i wizualizacji.

2. Załadowanie danych: Dane dotyczące warunków pogodowych i produkcji energii zostały załadowane z pliku CSV. 

3. Dane dla stolic Europy: Stworzono dodatkowy zbiór danych zawierający informacje o lokalizacjach stolic europejskich, które będą wykorzystywane do wizualizacji na mapie.

4. Inicjalizacja aplikacji Dash: Aplikacja Dash została zainicjowana, a jej layout został zdefiniowany. Layout zawiera elementy wejściowe (pole tekstowe do wprowadzania nazwy miasta, przycisk do wysyłania zapytania, suwak do wyboru zakresu lat) oraz elementy wyjściowe (wykresy i mapa).

5. Definiowanie callbacków: Callbacki w Dashu umożliwiają dynamiczną aktualizację wykresów i mapy w odpowiedzi na interakcje użytkownika. W tym przypadku, callback obsługuje wprowadzanie nazwy miasta, kliknięcie przycisku oraz wybór zakresu lat.

6. Logika aplikacji: W funkcji callback zaimplementowano logikę filtrowania danych na podstawie wejść użytkownika, a następnie tworzenie i aktualizowanie wykresów oraz mapy. Dodatkowo, zawiera ona obliczenia szacowanego rocznego zapotrzebowania na energię dla domów o różnych rozmiarach i typach gospodarstw domowych.

In [11]:
# Pandas jest używany do manipulacji i analizy danych.
import pandas as pd

# Importowanie niezbędnych modułów z biblioteki Dash do tworzenia interaktywnych aplikacji internetowych.
import dash
from dash import dcc, html, Input, Output, State

# Ponowne zaimportowanie pandas, co jest zbędne, ponieważ pandas zostało już zaimportowane wcześniej.
import pandas as pd

# Importowanie bibliotek Plotly do tworzenia wykresów i wizualizacji danych.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objects import Bar, Figure

In [12]:
# Wczytywanie danych
df_weather_final = pd.read_csv("../2. Data/processed/final_df_processed.csv")

In [13]:
# Wyświetla pierwsze 10 wierszy DataFrame df_weather_final
df_weather_final.head(10)

,Unnamed: 0,date,city_name,lat,lon,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,clouds_all,weather_main,estimated_irradiance,temp_modifier,humidity_modifier,daily_energy_production_kWh,year,month
0,0,1979-01-01,Baden_Baden,48.765640,8.228524,-10.442083,-16.532917,-16.18,3.02,1013.666667,81.916667,3.681667,84.500000,Cloudy,2.0,1.0,0.95,7.6,1979,1
1,1,1979-01-01,Bozanska,54.473028,18.302179,-14.100000,-20.617500,-19.14,-9.46,1002.416667,82.000000,6.157917,100.000000,Cloudy,2.0,1.0,0.95,7.6,1979,1
2,2,1979-01-01,Walcz_Drugi,53.257039,16.521203,-16.437083,-23.322500,-21.26,-9.57,1003.708333,82.416667,3.682083,94.791667,Cloudy,2.0,1.0,0.95,7.6,1979,1
3,3,1979-01-02,Baden_Baden,48.765640,8.228524,-8.856667,-14.874167,-15.82,-3.96,1024.250000,84.208333,4.524583,71.708333,Snow,1.5,1.0,0.95,5.7,1979,1
4,4,1979-01-02,Bozanska,54.473028,18.302179,-9.037917,-15.179167,-14.14,-7.40,1004.791667,85.375000,4.449583,99.541667,Cloudy,2.0,1.0,0.95,7.6,1979,1
5,5,1979-01-02,Walcz_Drugi,53.257039,16.521203,-7.259583,-14.093333,-10.04,-5.30,1008.500000,88.458333,5.382500,99.458333,Cloudy,2.0,1.0,0.95,7.6,1979,1
6,6,1979-01-03,Baden_Baden,48.765640,8.228524,-7.245417,-11.859583,-12.99,-2.53,1023.833333,83.333333,2.877917,54.541667,Clouds,2.5,1.0,0.95,9.5,1979,1
7,7,1979-01-03,Bozanska,54.473028,18.302179,-10.780000,-16.679167,-16.33,-6.84,1004.500000,87.041667,3.805833,97.083333,Cloudy,2.0,1.0,0.95,7.6,1979,1
8,8,1979-01-03,Walcz_Drugi,53.257039,16.521203,-8.548750,-15.455417,-11.76,-6.76,1007.625000,87.416667,5.291250,97.791667,Cloudy,2.0,1.0,0.95,7.6,1979,1
9,9,1979-01-04,Baden_Baden,48.765640,8.228524,-8.997083,-15.285417,-13.37,-4.80,1012.291667,81.333333,4.802500,77.125000,Clouds,2.5,1.0,0.95,9.5,1979,1


In [14]:
# Załadowanie danych z pliku CSV i przekształcenie kolumny 'date' na typ datetime
df = pd.read_csv("../2. Data/processed/final_df_processed.csv")
df['date'] = pd.to_datetime(df['date']) # Konwersja kolumny 'date' na format daty
df['year'] = df['date'].dt.year # Wyciągnięcie roku z kolumny 'date'
df['month'] = df['date'].dt.month # Wyciągnięcie miesiąca z kolumny 'date'

# Dane dla stolic Europy w formie słownika
capitals_data = {
    'city_name': ['Berlin', 'Paris', 'Madrid', 'Rome', 'Warsaw', 'London', 'Athens', 'Vienna', 'Budapest', 'Prague', 'Moscow', 'Kiev', 'Oslo', 'Stockholm', 'Helsinki', 'Copenhagen', 'Amsterdam', 'Brussels', 'Lisbon', 'Dublin'],
    'lat': [52.52, 48.8566, 40.4165, 41.9028, 52.2297, 51.5074, 37.9838, 48.2082, 47.4979, 50.0755, 55.7558, 50.4501, 59.9139, 59.3293, 60.1695, 55.6761, 52.3676, 50.8503, 38.7223, 53.3498],
    'lon': [13.405, 2.3522, -3.7038, 12.4964, 21.0122, -0.1278, 23.7275, 16.3738, 19.0402, 14.4378, 37.6173, 30.5234, 10.7522, 18.0686, 24.9384, 12.5683, 4.9041, 4.3517, -9.1393, -6.2603]
}
# Konwersja słownika na DataFrame
capitals_df = pd.DataFrame(capitals_data)

# Inicjalizacja aplikacji Dash
app = dash.Dash(__name__)

# Definiowanie układu aplikacji
app.layout = html.Div([
    dcc.Input(id='city-input', type='text', placeholder='Wpisz nazwę miejscowości'),  # Pole tekstowe do wprowadzania nazwy miasta
    html.Button('Sprawdź', id='submit-button', n_clicks=0),  # Przycisk do wysyłania zapytania
    html.Div("---------------------------------------------"),
    html.Div("Otwórz w przeglądarce: http://127.0.0.1:8050/"),
    html.Div("---------------------------------------------"),
    html.Div("Dostępne miejscowości: Baden_Baden, Bozanska, Walcz_Drugi"),  # Lista dostępnych miast
    html.Div(id='error-message'),  # Miejsce na komunikaty o błędach
    html.Div([
        dcc.Graph(id='monthly-weather-chart', style={'width': '48%','height': '100vh', 'display': 'inline-block', 'margin-right': '2%'}),  # Wykres miesięcznych warunków pogodowych
        dcc.Graph(id='map-chart', style={'width': '48%','height': '100vh', 'display': 'inline-block', 'margin-left': '2%'})  # Mapa z lokalizacją miasta
    ]),
    html.Div([
        dcc.Graph(id='monthly-energy-chart', style={'width': '100%'})  # Wykres miesięcznej produkcji energii
    ]),
    html.Div([
        dcc.Graph(id='annual-energy-chart', style={'width': '100%', 'height': '400px'}),  # Wykres rocznej produkcji energii
        dcc.RangeSlider(
            id='year-range-slider',
            min=df['year'].min(),
            max=df['year'].max(),
            value=[df['year'].min(), df['year'].max()],
            marks={str(year): str(year) for year in df['year'].unique()},
            step=None,
            tooltip={"placement": "bottom", "always_visible": True}  # Konfiguracja Tooltip
        )
    ], style={'width': '100%'}),
    
    # Div z informacjami o "Szacowanym zapotrzebowaniu"
    html.Div([
        html.H3("Szacowane zapotrzebowanie:"),
        html.Ol([
            html.Li([
                html.H4("Wyznaczenie podstawowych wartości zużycia energii dla pomp ciepła:"),
                html.P("Pompy ciepła są zazwyczaj bardziej efektywne niż tradycyjne systemy ogrzewania, więc ich zużycie energii jest niższe. Przyjmując, że pompa ciepła ma średnią efektywność i biorąc pod uwagę różne rozmiary domów, można założyć następujące wartości:"),
                html.Ul([
                    html.Li("70 m²: Około 50 kWh/m² rocznie"),
                    html.Li("100 m²: Około 50 kWh/m² rocznie"),
                    html.Li("150 m²: Około 50 kWh/m² rocznie")
                ])
            ]),
            html.Li([
                html.H4("Kalkulacja rocznego zapotrzebowania na energię:"),
                html.P("Dla rodziny z dwojgiem dzieci:"),
                html.Ul([
                    html.Li("70 m²: 70 m² × 50 kWh/m² = 3500 kWh rocznie"),
                    html.Li("100 m²: 100 m² × 50 kWh/m² = 5000 kWh rocznie"),
                    html.Li("150 m²: 150 m² × 50 kWh/m² = 7500 kWh rocznie")
                ]),
                html.P("Dla pary (założenie: mniejsze zapotrzebowanie na ciepłą wodę i ogólnie energię):"),
                html.Ul([
                    html.Li("70 m²: 70 m² × 50 kWh/m² × 0.85 = 2975 kWh rocznie (15% mniej)"),
                    html.Li("100 m²: 100 m² × 50 kWh/m² × 0.85 = 4250 kWh rocznie"),
                    html.Li("150 m²: 150 m² × 50 kWh/m² × 0.85 = 6375 kWh rocznie")
                ])
            ]),
            html.Li([
                html.H4("Uwagi:"),
                html.Ul([
                    html.Li("Przyjęto, że zapotrzebowanie na energię jest niższe o 15% dla pary w porównaniu z rodziną z dwojgiem dzieci ze względu na mniejsze zużycie ciepłej wody i mniejszą liczbę urządzeń elektrycznych w użytku."),
                    html.Li("Rzeczywiste zużycie energii może się różnić w zależności od jakości izolacji domu, lokalizacji geograficznej (klimatu), stylu życia mieszkańców oraz wieku i stanu technicznego pompy ciepła.")
                ])
            ]),
            html.Li([
                html.H4("Wnioski:"),
                html.P("Te przybliżone obliczenia dają ogólny obraz potencjalnego zapotrzebowania na energię w domach ogrzewanych za pomocą pompy ciepła. Warto jednak zauważyć, że indywidualne warunki mogą znacząco wpłynąć na rzeczywiste zużycie energii. Dla dokładniejszej analizy i optymalizacji zużycia energii zalecane jest przeprowadzenie audytu energetycznego domu.")
            ])
        ])
    ])
])

# Callback dla dynamicznej aktualizacji i mapy
@app.callback(
    [Output('error-message', 'children'), # Wyjście dla komunikatu o błędzie
     Output('city-input', 'style'), # Wyjście dla stylu pola tekstowego
     Output('annual-energy-chart', 'figure'), # Wyjście dla rocznego wykresu produkcji energii
     Output('monthly-weather-chart', 'figure'), # Wyjście dla wykresu miesięcznych warunków pogodowych
     Output('map-chart', 'figure'), # Wyjście dla mapy
     Output('monthly-energy-chart', 'figure')],  # Wyjście dla wykresu miesięcznej produkcji energii
    [Input('submit-button', 'n_clicks'), # Wejście dla liczby kliknięć przycisku
     Input('year-range-slider', 'value')], # Wejście dla zakresu lat z suwaka
    [State('city-input', 'value')] # Wejście dla wartości z pola tekstowego
)
def update_output(n_clicks, year_range, city):
    # n_clicks w funkcji update_output jest wartością liczbową, która wskazuje, ile razy przycisk został kliknięty. Jego główną rolą jest umożliwienie wywołania funkcji update_output za każdym razem, gdy przycisk zostanie kliknięty, co pozwala na dynamiczną aktualizację zawartości aplikacji Dash w odpowiedzi na akcję użytkownika.

    # Inicjacja pustych wykresów dla przypadku błędu
    empty_fig = {'data': [], 'layout': {'title': {'text': 'Brak danych do wyświetlenia'}}}

    # Sprawdzanie, czy podane miasto istnieje w danyc
    if city not in df['city_name'].unique():
        # Zwraca pustą mapę, wykresy i komunikat o błędzie
        return ('Nie ma takiej miejscowości w bazie', {'border': '2px solid red'}, empty_fig, empty_fig, empty_fig, empty_fig)
    
    # Filtrowanie danych dla wybranego miasta i zakresu lat
    filtered_df = df[(df['city_name'] == city) & (df['year'] >= year_range[0]) & (df['year'] <= year_range[1])]
    # Roczna produkcja energii w postaci wykresu słupkowego
    annual_energy_fig = px.bar(filtered_df, x='year', y='daily_energy_production_kWh', title=f'Roczna produkcja energii przez lata dla {city}')

    # Miesięczne warunki pogodowe w postaci wykresów liniowych
    weather_attributes = ['temp','clouds_all', 'humidity', 'wind_speed']
    attribute_labels = ['Temperatura (°C)','Zachmurzenie (%)', 'Wilgotność (%)', 'Prędkość wiatru (m/s)']
    monthly_weather_fig = make_subplots(rows=4, cols=1, subplot_titles=attribute_labels, vertical_spacing=0.2)
    for i, attr in enumerate(weather_attributes, start=1):
        monthly_data = filtered_df.groupby('month')[attr].mean().reset_index()
        monthly_weather_fig.add_trace(go.Scatter(x=monthly_data['month'], y=monthly_data[attr], mode='lines+markers'), row=i, col=1)
        monthly_weather_fig.update_xaxes(title_text="Miesiące", tickvals=list(range(1, 13)), row=i, col=1)

    monthly_weather_fig.update_layout(height=800, showlegend=False, title_text="Średnie miesięczne warunki pogodowe na przestrzeni wszystkich lat", title_x=0.5)
    monthly_weather_fig.update_yaxes(rangemode='tozero')

    # Obliczanie średniej miesięcznej dla wybranego miasta bez ograniczenia lat
    city_data = df[df['city_name'] == city]
    grouped_data = city_data.groupby(['year', 'month'])['daily_energy_production_kWh'].sum().reset_index()
    average_monthly_energy = grouped_data.groupby('month')['daily_energy_production_kWh'].mean().reset_index()

    # Tworzenie wykresu Plotly
    monthly_energy_fig = Figure(data=[
        Bar(x=average_monthly_energy['month'], y=average_monthly_energy['daily_energy_production_kWh'])
    ])
    # Tworzenie wykresu dla średniej miesięcznej produkcji energii
    monthly_energy_fig.update_layout(
        title=f'Średnia miesięczna produkcja energii przez miesiące dla {city} na przestrzeni lat',
        xaxis=dict(title='Miesiąc', tickmode='array', tickvals=list(range(1, 13)), ticktext=['Sty', 'Lut', 'Mar', 'Kwi', 'Maj', 'Cze', 'Lip', 'Sie', 'Wrz', 'Paź', 'Lis', 'Gru']),
        yaxis=dict(title='Średnia miesięczna produkcja energii (kWh)'),
        barmode='group'
    )

        # Znajdź dane dla wybranej lokalizacji
    location_data = df[df['city_name'] == city].drop_duplicates(subset=['city_name'])

    # Zamień "trace 1" na "Stolice" w danych lokalizacji
    location_data['city_name'] = location_data['city_name'].replace('trace 1', 'Stolice')

    # Tworzenie wykresu dla wybranej lokalizacji
    map_fig = px.scatter_geo(location_data, lat='lat', lon='lon', hover_name='city_name',
                            projection='natural earth', title='Lokalizacja na mapie Europy',
                            size_max=20, color_discrete_sequence=["red"])

    # Zamień "trace 1" na "Stolice" w danych stolic
    capitals_df['city_name'] = capitals_df['city_name'].replace('trace 1', 'Stolice')

    # Dodanie punktów dla Stolic z odpowiednią etykietą
    map_fig.add_scattergeo(lat=capitals_df['lat'], lon=capitals_df['lon'], hoverinfo='text',
                        text=capitals_df['city_name'], marker=dict(size=5, color='blue', symbol='circle'), name='Stolice')

    # Dodanie śladu dla wybranej lokalizacji z odpowiednią etykietą
    map_fig.add_scattergeo(lat=location_data['lat'], lon=location_data['lon'], hoverinfo='text',
                        text=city, marker=dict(size=10, color='red', symbol='circle'),
                        name=city)

    # Aktualizacja ustawień mapy
    map_fig.update_geos(
        visible=True, showcountries=True, countrycolor="Black",
        showcoastlines=True, coastlinecolor="Black",
        showland=True, landcolor="LightGreen",
        projection_scale=2
    )
    map_fig.update_layout(
        geo=dict(
            scope='europe', projection_type='orthographic', showframe=False,
            showcoastlines=True, landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',
            lataxis_range=[35, 70], 
            lonaxis_range=[-25, 40], 
            projection_scale=2  
        ),
        legend_title_text='Legenda'
    )

    return '', {'border': '1px solid green'}, annual_energy_fig, monthly_weather_fig, map_fig, monthly_energy_fig

# Uruchomienie serwera aplikacji Dash
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)  # 


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

c:\Users\reszk\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
